In [3]:
import numpy as np
from theano import function
import theano.tensor.slinalg
import theano.tensor as T

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import timeit

rng = np.random

In [86]:
class mDA(object):
    def __init__(self, input):
        self.input = input
        
        self.W = T.dmatrix("W")
        self.Xh = T.dmatrix("Xh")
        
        self.output = T.tanh(T.dot(self.W, self.Xh))
        
    
    def get_mapping(self, pr):
        X = T.transpose(self.input)
        
        # X=[X;ones(1,size(X,2))];
        X = T.concatenate([X, T.ones((1, X.shape[1]))], axis=0)
        
        #d=size(X,1);
        d = X.shape[0]
        
        #q=[ones(d-1,1).*(1-p); 1];
        q = T.concatenate([T.ones((d-1,1)) * (1-pr), T.ones((1,1))], axis=0)
        
        #S=X*X’;
        S = T.dot(X, X.T)
        
        #Q=S.*(q*q’);
        Q = S * T.dot(q, q.T)
        
        #Q(1:d+1:end)=q.*diag(S);
        Q -= (T.eye(Q.shape[0]) * Q.diagonal())
        Q += T.eye(Q.shape[0]) * T.diagonal(q*S.diagonal())
        
        #P=S.*repmat(q’,d,1);
        P = S * T.extra_ops.repeat(q.T, d, 0)
        
        #W=P(1:end-1,:)/(Q+1e-5*eye(d));
        
        A = Q + 10**-5*T.eye(d)
        B = P
        
        self.W = T.slinalg.solve(A.T,B.T)[:-1,:]
        self.Xh = T.tanh(T.dot(self.W, X)).T
        
        return self.W, self.Xh
    
    def predict(self, X, W):
        X = T.concatenate([X.T, T.ones((1, X.T.shape[1]))], axis=0)
        #return T.tanh(T.dot(mapping, X))
        pred = T.tanh(T.dot(W, X))
        return pred.T


class mSDA(object):
    def __init__(self, input, l, pr):
        self.input = input
        self.l = l
        self.pr = pr
        
        self.mDA_layers = []
        self.W_layers = []
        
        for i in xrange(self.l):
            if i==0:
                layer_input = self.input
            else:
                layer_input = self.mDA_layers[-1].output
        
            mda = mDA(layer_input)
            
            self.mDA_layers.append(mda)
    
    def train_fns(self):
        fns = []
        for mda in self.mDA_layers:
            W, Xh = mda.get_mapping(self.pr)
            train_mda = function(
                [mda.input],
                [W, Xh]
            )
            
            fns.append(train_mda)
        
        return fns
    
    def fit(self, X):
        training_fns = self.train_fns()
        
        start_time = timeit.default_timer()

        inputs = [X]

        i = 1
        for fn in training_fns:
            print "Entrenando capa ", i
            W, Xh = fn(inputs[-1])

            inputs.append(Xh)

            self.W_layers.append(W)

            i += 1

        end_time = timeit.default_timer()

        tiempo = (end_time - start_time)/60.
        print "Entrenado en %.2fm\n" % (tiempo)

        return tiempo
    
    def predict(self, X):
        Xhs = [X]
        
        for i in xrange(self.l):
            mda = self.mDA_layers[i]
            W = self.W_layers[i]

            x2 = T.dmatrix('x')
            W2 = T.dmatrix('w')

            y = mda.predict(x2, W2)
            
            
            pred = function(
                [x2, W2],
                y
            )
            
            Xh = pred(Xhs[i], W)
            Xhs.append(Xh)
        
        return Xhs[-1]

# Entrenar solo el mda

In [82]:
x = T.dmatrix('x')
# Dataset ficticio formado por 1000 ejemplos de dimension 500

samples = 10000
n_test = 400
features = 3000

D_train = (rng.randint(0,2, (samples, features)), rng.randint(0,2,(samples, )))
D_test = (rng.randint(0,2, (n_test, features)), rng.randint(0,2,(n_test, )))

D_all = np.vstack([D_train[0], D_test[0]])


l = 3
pr = 0.7


In [83]:
print D_train[0].shape
print D_all.shape

(10000, 3000)
(10400, 3000)


In [62]:
mda = mDA(x)

W, Xh = mda.get_mapping(0.5)
train_mda = function(
    [mda.input],
    [W, Xh]
)




In [63]:
Wr, Xhr = train_mda(D_train[0])

In [64]:

Xhr.shape

(10, 20)

In [65]:
Wn = T.dmatrix('Wn')
Xn = T.dmatrix('Xn')

y = mda.predict(W=Wn, X=Xn)
#y = T.concatenate([Xn.T, T.ones((1, Xn.T.shape[1]))], axis=0)

predict = function(
    [Xn,Wn],
    y
)

In [66]:
print D_test[0].shape
predict(D_test[0], Wr).shape

(40, 20)


(40, 20)

# Entrenar el mSDA

In [84]:
msda = mSDA(x, l, pr)
#training_fns = msda.train_fns()
msda.fit(D_all)

Entrenando capa  1
Entrenando capa  2
Entrenando capa  3
Entrenado en 7.64m



7.637819584210714

In [92]:
training_fns = msda.train_fns()
        
start_time = timeit.default_timer()

inputs = [D_all]

i = 1
for fn in training_fns:
    print "Entrenando capa ", i
    W, Xh = fn(inputs[-1])

    inputs.append(Xh)

    msda.W_layers.append(W)

    i += 1

end_time = timeit.default_timer()

tiempo = (end_time - start_time)/60.
print "Entrenado en %.2fm\n" % (tiempo)

Entrenando capa  1
Entrenando capa  2
Entrenando capa  3
Entrenado en 7.17m



In [85]:
X_new = msda.predict(D_test[0])

W:  (3000, 3001)
Pred:  (400, 3000)
W:  (3000, 3001)
Pred:  (400, 3000)
W:  (3000, 3001)
Pred:  (400, 3000)


In [87]:
print D_all.shape
print X_new.shape

(10400, 3000)
(400, 3000)


In [88]:


tuned_parameters = [{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]


clf1 = GridSearchCV(SVC(), tuned_parameters, n_jobs=4)
clf2 = GridSearchCV(SVC(), tuned_parameters, n_jobs=4)

In [89]:
print D_test[0].shape
print X_new.shape
print D_test[1].shape

(400, 3000)
(400, 3000)
(400,)


In [91]:
clf1 = clf1.fit(D_test[0], D_test[1])
print clf1.best_score_

clf2 = clf2.fit(X_new, D_test[1])
print clf2.best_score_



0.545
0.49
